# Popular_Tweet_Bot

The next cell will authenticate your bot based on your keys and tokens. See the [Tweepy tutorial](https://github.com/comp-journalism/UMD-J479V-J779V-Spring2016/blob/master/Weekly/Week_5/Tweepy-skeleton.ipynb) for more details. 

In [15]:
import warnings
warnings.filterwarnings('ignore')

import re
import tweepy
import pandas as pd
import numpy as np
import csv

pd.set_option('display.max_columns', None)

#Setup and authenticate Tweepy
CONSUMER_KEY = "##################"
CONSUMER_SECRET = "##################"
ACCESS_TOKEN = "##################"
ACCESS_TOKEN_SECRET = "##################"

#Constant Text
Tweet_Error_Message = "Include hastag to indicate search phrase, Format: #Search phrase#"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)
# added wait on rate limit so tweepy can handle rate limit automatically

me = api.me()
print (me.screen_name)


Rate_Tweets_Bot


The next cell has a number of essential functions that allow your bot to listen to the twitter stream and then respond. The assignment involves fleshing out the editorial logic of the bot and writing code in the sections marked "TODO".

In [16]:
import random
import json
import time

# If debug_mode is True then the bot won't actually tweet. Set debug_mode = False for it to tweet. 
debug_mode = False

#################################################################
# Functions that allow the bot to tweet or reply to tweets
def tweet(status):
    print ("JUST TWEETED: ", status)
    # Only *actually* send the tweet on twitter if we're not in debug mode
    if debug_mode == False:
        api.update_status(status)
#################################################################       
def tweet_with_probability(status, probability):
    # Change the probability of tweeting variable to affect how often the bot tweets
    rand = random.random()
    if rand <= probability:
        print ("JUST TWEETED: ", status)
        # Only *actually* send the tweet on twitter if we're not in debug mode
        if debug_mode == False:
            api.update_status(status)
#################################################################       
def tweet_reply(status, tweet_to_reply_to):
    screen_name = tweet_to_reply_to["user"]["screen_name"]
    at = ".@"+screen_name
    reply_to_id = tweet_to_reply_to["id"]
    status = at + " " + status
    print ("JUST TWEETED: ", status)
    # To actually have the bot tweet a status message uncomment the next line, 
    #it will send the tweet in response to the appropriate tweet. NOTE: be careful
    #about spamming other people by reply to them. 
    # Only *actually* send the tweet on twitter if we're not in debug mode
    if debug_mode == False:
        api.update_status(status=status, in_reply_to_status_id=reply_to_id)

In [17]:
#################################################################
# We derive a class that is used to listen to the twitter stream
class CustomTwitterStreamListener(tweepy.StreamListener):
    def __init__(self):
        super(CustomTwitterStreamListener, self).__init__()
        self.num_tweets = 0
        self.api_count = 0
        self.old_time = time.time()
    
    # Everytime the listener encounters a tweet that matches its filters it will trigger the on_data function
    def on_data(self, data):
        try:

            # The data variable represents the tweet that was detected by the stream listener
            # Here we just parse it as JSON and put it in another variable. 
            tweet_data = json.loads(data)
            
            # Make sure the bot never interacts with itself
            user = tweet_data["user"]["screen_name"]
            user_mentions = tweet_data['entities']['user_mentions']
            user_mentions_list = []
            for u in user_mentions:
                user_mentions_list.append(u["screen_name"])
            if user == me.screen_name or me.screen_name in user_mentions_list:
                # return without doing anything
                return

            # Skip Retweets
            if  tweet_data['retweeted'] or tweet_data['text'].startswith('RT'):
                return
            else:
                print ("Found Tweet")
            
            self.num_tweets = self.num_tweets + 1
            
            #print (tweet_data["text"])
            global collection_Tweets 
            collection_Tweets.append(tweet_data)
            if self.num_tweets >= 100: #Limiting the tweets collected
                return false
           
        except Exception as e:
            print ("exception: " + e)
            pass
        
        return "Hello" 

In [18]:
#################################################################
#Method to get the last tweets
#This Method checks if there is any tweets that has mentioned the newsbot
#If found it parses the tweet to understand the request
#This method picks up all tweets till the last tweet that was processed
def checkForMention():
    print ("Quering NewsBot's Mentions Timeline")
    mentions = api.mentions_timeline(count=1,since_id=lastTweet_Id) #Getting last one tweet for development purpose
    
    for mention in mentions:
        # The mention variable represents the tweet that sent to the bot
        # Parse and store the data it in another variable. 
        mentioned_tweet = mention._json
        customer_Name = mentioned_tweet["user"]["screen_name"]
        tweet_id = mentioned_tweet["id"]
        
        #Parse the tweet to get the phrase and twitter id
        parsed_Value=parseTweets(mentioned_tweet)
        
        no_accounts = len(parsed_Value["user_ids"])
        no_tags = len(parsed_Value["search_tags"])
        
        #If user request to search for phase in list of accounts or single account
        if no_accounts > 0 and no_tags > 0: 
            gatherFollowerTweets(parsed_Value["user_ids"]);
            CreateRatings(parsed_Value["search_tags"],mentioned_tweet);
            
        elif no_tags > 0: #If user request to search for phase all accounts
            gatherTweets(parsed_Value["search_tags"]);
            CreateRatings(parsed_Value["search_tags"],mentioned_tweet);
            print ('User:{} Requested to search Phrase:"{}" all accounts'.format(customer_Name,
                                                                                parsed_Value["search_tags"]))
        elif no_accounts > 0: #If user request to rate tweets of a particular account
            gatherFilteredTweets(parsed_Value["user_ids"]);
            CreateRatings_User(parsed_Value["search_tags"],mentioned_tweet);
            print ('User:{} Requested to search all tweets in {}'.format(customer_Name,
                                                                                parsed_Value["user_ids"]))               
        else: #Search best news from predfined tweet accounts        
            gatherFilteredTweets(parsed_Value["user_ids"]);
            CreateRatings_User(parsed_Value["search_tags"],mentioned_tweet);
            print ('User:{} Requested to search all tweets'.format(customer_Name))  
        
        
        global lastTweet_Id 
        lastTweet_Id = tweet_id 
        #Updating Last tweet id so bot will not pick up old tweets
    
    
    print ("Finished instance - Checking Mentions Timeline") 
    
    return

In [19]:
#################################################################
#function to parse tweets and identify the phrase and accounts listed in tweets
def parseTweets(tweet_json):  
    
    tweet = tweet_json["text"] #Get the tweet to parse the text
    #print (tweet)

    user_ids= {tweet.strip("@") for tweet in tweet.split() if tweet.startswith("@")}# Get list of accounts in tweet
    user_ids.remove(me.screen_name) #remove bot id

    tags = re.search(r'\#(.*)\#', tweet)
    if tags is None:
        search_tags=''
    else:
        search_tags=tags.group(1)
    
    #List the phrase that needs to be searched    
    return {'user_ids':user_ids, 'search_tags':search_tags}

In [20]:
#################################################################
#Function to gather and filter next 100 tweets for pharse - Using Streaming api
def gatherTweets(search_tags):
    print (search_tags)
    cust_listener = CustomTwitterStreamListener()
    cust_stream = tweepy.Stream(auth, cust_listener)
    
    # track_object filters the twitter stream to capture data, can provide a hashtag, word, 
    #or screenname amongths other things. 
    #See documentation at: https://dev.twitter.com/streaming/overview/request-parameters
    # Currently set to track all tweets mentioning nytimes.com
    track_tag = [search_tags]
    try:
        # Start filtering the twitter stream
        cust_stream.filter(track=track_tag)
    except:
        # do nothing in the event of an error
        pass
    return

In [21]:
#################################################################
#Function to gather and filter last 100 tweets for pharse from a list of followers
def gatherFollowerTweets(user_names,search_tags):
    print (user_names)
    print (search_tags)
    cust_listener = CustomTwitterStreamListener()
    cust_stream = tweepy.Stream(auth, cust_listener)
    
    user_ids = []
    
    for user in user_names:
        user_id = getUserId(user)
        user_ids.append(user_id)
        
    followers = api.friends_ids(screen_name=me.screen_name)
    final_list = followers+user_ids
        
    follow_id = [user_ids]
    track_tag = [search_tags]

    try:
        # Start filtering the twitter stream
        cust_stream.filter(follow = follow_id)
        cust_stream.filter(track=track_tag)

    except:
        # do nothing in the event of an error
        pass
    return
#################################################################
#Function to gather and filter last 50 tweets for one or more user
def gatherFilteredTweets(user_names):
    
    user_ids = []
    for user in user_names:
        timeline = api.user_timeline(screen_name=user, include_rts=False, count=200)  
        for status in timeline:
            global collection_Tweets 
            collection_Tweets.append(status._json) 
            
    followers = api.friends_ids(screen_name=me.screen_name)
    for user in followers:
        timeline = api.user_timeline(id=user, include_rts=False, count=200)  
        for status in timeline:
            global collection_Tweets 
            collection_Tweets.append(status._json) 
            

    return

#Function to get user id from userName
def getUserId(screenName):
    user = api.get_user(screen_name=screenName)
    return user.id

In [22]:
#################################################################
#function that cleanupdate into dataframe format to calculate ratings
def CleanUpData():
    
    tweet_table = []
    for tweets in collection_Tweets:
        tweet_row = []
        
        #Creating a datatable to perform some functions
        tweet_row.append(tweets["text"])
        tweet_row.append(tweets["retweet_count"])
        tweet_row.append(tweets["favorite_count"])
        
        #Creating list of other hashtags in the tweet
        hastags_json = tweets["entities"]["hashtags"]
        hastags = []
        for hastag in hastags_json:
            hastags.append(hastag["text"])        
        #append the list of hastags to the datatable
        tweet_row.append(', '.join(hastags))
        tweet_row.append(tweets["user"]["followers_count"])
        tweet_row.append(tweets["user"]["favourites_count"])
        tweet_row.append(tweets["user"]["location"])
        tweet_row.append(tweets["user"]["verified"])
        #Logic to rate the tweets based on parameters
        
        Rating = 0
        if tweets["user"]["verified"] == 'True':
            Rating = 500
        tweet_row.append(Rating)
        tweet_row.append(tweets["user"]["screen_name"])
        tweet_row.append(tweets["id_str"])
        
        #Create new row
        tweet_table.append(tweet_row)
    
    #Creating dataframe from the list of tweets to perform manipulation
    tweet_df = pd.DataFrame(tweet_table, columns=['Tweet', 'Retweet Count', 'Favourite Count', 'Hashtags',
                                                 'Followers Count','Favourites Count','Location',
                                                  'Is Verified?','Rating','User Name','Tweet ID'])
    
    return tweet_df

In [23]:
#################################################################
#Function to create ratings and rank the tweets based on ratings
#Top 3 tweets are tweeted back to the users
# High Rating is given to a verified user, User with high followers count and then based total favorite counts
def CreateRatings(searchTag,mentioned_tweet):
    tweets_csv = CleanUpData()
    
    tweets_csv["Rating"] = (1 * tweets_csv["Rating"] + ((0.50*tweets_csv["Followers Count"]) +
                            (0.10 * tweets_csv["Retweet Count"]) + 
                            (0.10 * tweets_csv["Favourite Count"]) + (0.30 * tweets_csv["Favourites Count"])))
    tweets_csv = tweets_csv.sort_values(by="Rating", ascending=False)

    from IPython.display import display, HTML
    display(tweets_csv)
        
    if len(tweets_csv.index) > 0:
        #Pick the first 3 tweet
        sendTopTweets(tweets_csv.iloc[2],searchTag,mentioned_tweet,1)
        sendTopTweets(tweets_csv.iloc[1],searchTag,mentioned_tweet,2)
        sendTopTweets(tweets_csv.iloc[0],searchTag,mentioned_tweet,3)

    return

#################################################################
#Function to create ratings and rank the tweets based on ratings
#Top 3 tweets are tweeted back to the users
# High Rating is given to tweets that are retweeted and favoured more then to followers count
def CreateRatings_User(searchTag,mentioned_tweet):
    tweets_csv = CleanUpData()
    
    tweets_csv["Rating"] = (1 * tweets_csv["Rating"] + ((0.40*tweets_csv["Favourite Count"]) +
                            (0.60 * tweets_csv["Retweet Count"])+(0.05 * tweets_csv["Followers Count"])))
    tweets_csv = tweets_csv.sort_values(by="Rating", ascending=False)

    from IPython.display import display, HTML
    display(tweets_csv)
        
    if len(tweets_csv.index) > 0:
        #Pick the first 3 tweet
        sendTopTweets_user(tweets_csv.iloc[2],searchTag,mentioned_tweet,1)
        sendTopTweets_user(tweets_csv.iloc[1],searchTag,mentioned_tweet,2)
        sendTopTweets_user(tweets_csv.iloc[0],searchTag,mentioned_tweet,3)

    return

#################################################################
#Function to format the tweets 
def sendTopTweets_user(Tweet,searchTag,mentioned_tweet,rank):
  
    TweetMessage = "Tweet {}: https://twitter.com/frjoemungai/status/{} ,Engagment Rating:{}".format(rank,
                                                Tweet["Tweet ID"],int(Tweet["Rating"]))
    print (len(TweetMessage))
    print (TweetMessage)

    tweet_reply(TweetMessage,mentioned_tweet)

    return
#################################################################
#Function to format the tweets 
def sendTopTweets(Tweet,searchTag,mentioned_tweet,rank):
    
    TweetMessage = "Tweet {}: https://twitter.com/frjoemungai/status/{} ,Potential Audience:{}, Rating:{}".format(rank,
                                                Tweet["Tweet ID"],Tweet["Followers Count"],int(Tweet["Rating"]))
    print (len(TweetMessage))
    print (TweetMessage)

    tweet_reply(TweetMessage,mentioned_tweet)

    return

In [25]:
lastTweet_Id=712352387994947585 #Initial Tweet id for development

collection_Tweets=[]#Initialize the collection
checkForMention()#Main Method
print ("Total Number of Tweets collected: {}".format(len(collection_Tweets)))

Quering NewsBot's Mentions Timeline


,Tweet,Retweet Count,Favourite Count,Hashtags,Followers Count,Favourites Count,Location,Is Verified?,Rating,User Name,Tweet ID
892,Texas A&amp;M: Down 12 with 35 sec. to go in r...,5174,6321,,25937205,127,"Bristol, CT",True,1302493.05,espn,711748010099806208
926,Warriors have now lost 33 straight games in Sa...,3652,4694,,25937205,127,"Bristol, CT",True,1300929.05,espn,711386805837365249
967,Madness at its finest!\n\nThis is the first ti...,3246,4595,,25937205,127,"Bristol, CT",True,1300645.85,espn,711013234346737664
971,"On this day 21 years ago, a fax was sent with ...",2914,5019,,25937205,127,"Bristol, CT",True,1300616.25,espn,710987913178124288
962,AT THE BUZZER!!\n\nNorthern Iowa drills a shot...,2849,4964,,25937205,127,"Bristol, CT",True,1300555.25,espn,711044604615856128
961,History!\n\nNorthern Iowa's win over Texas was...,2371,4708,,25937205,127,"Bristol, CT",True,1300166.05,espn,711046612110462976
924,"On this day 50 yrs ago, Texas Western won it a...",2397,3821,,25937205,127,"Bristol, CT",True,1299826.85,espn,711396882677833730
944,"50 years ago today, Texas Western became the 1...",2469,3678,,25937205,127,"Bristol, CT",True,1299812.85,espn,711254686263672832
959,The thrill of victory. The agony of defeat.\n\...,1932,3928,,25937205,127,"Bristol, CT",True,1299590.65,espn,711051975530569732
937,Hoosier Hysteria!\n\nIndiana beats old rival K...,1981,3076,,25937205,127,"Bristol, CT",True,1299279.25,espn,711335361042063360


92
Tweet 1: https://twitter.com/frjoemungai/status/711013234346737664 ,Engagment Rating:1300645
JUST TWEETED:  .@Ramesh_br Tweet 1: https://twitter.com/frjoemungai/status/711013234346737664 ,Engagment Rating:1300645
92
Tweet 2: https://twitter.com/frjoemungai/status/711386805837365249 ,Engagment Rating:1300929
JUST TWEETED:  .@Ramesh_br Tweet 2: https://twitter.com/frjoemungai/status/711386805837365249 ,Engagment Rating:1300929
92
Tweet 3: https://twitter.com/frjoemungai/status/711748010099806208 ,Engagment Rating:1302493
JUST TWEETED:  .@Ramesh_br Tweet 3: https://twitter.com/frjoemungai/status/711748010099806208 ,Engagment Rating:1302493
User:Ramesh_br Requested to search all tweets
Finished instance - Checking Mentions Timeline
Total Number of Tweets collected: 5669
